#### **Q3: BUBBLE POINT METHOD – MULTICOMPONENT MULTISTAGE DISTILLATION** 
1000 kmol/h of a saturated-liquid mixture of 60% methanol (normal boiling point 65 C), 20 mol% ethanol 
(normal boiling point 98 C), and 20 mol% n-propanol (normal boiling point 97 C) is fed to the middle 
stage of a distillation column having three equilibrium stages, a total condenser, a partial reboiler, all 
operated at 1 atm. The distillate rate is 600 kmol/h, and the external reflux rate is 2,000 kmol/h of 
saturated liquid.

In [ ]:
#### **Q3: BUBBLE POINT METHOD – MULTICOMPONENT MULTISTAGE DISTILLATION** 
""" 
1000 kmol/h of a saturated-liquid mixture of 60% methanol (normal boiling point 65 C), 20 mol% ethanol 
(normal boiling point 98 C), and 20 mol% n-propanol (normal boiling point 97 C) is fed to the middle 
stage of a distillation column having three equilibrium stages, a total condenser, a partial reboiler, all 
operated at 1 atm. The distillate rate is 600 kmol/h, and the external reflux rate is 2,000 kmol/h of 
saturated liquid.
"""
# Write your own code to obtain converged solutions of the temperature profiles.


# Algorithm for obtaining converged solutions of the temperature profiles.
# 1. Assume an initial temperature profile.
# 2. Calculate the bubble point temperature of each stage.
# 3. Calculate the dew point temperature of each stage.
# 4. Calculate the average temperature of each stage.
# 5. Update the temperature profile.
# 6. Repeat steps 2-5 until the temperature profile converges.


In [ ]:
import numpy as np    
def calculate_Pij(A :np.array,B :np.array,C :np.array,T_j :np.array) -> np.array:
        """
        Calculate the saturation pressure of a component at a given temperature.

        Parameters:
        A : float
            Antoine's coefficient A.
        B : float
            Antoine's coefficient B.
        C : float
            Antoine's coefficient C.
        T : float
            Temperature in Celsius.

        Returns:
        float
            Saturation pressure in atm.
        """
        return np.array([[10**(A[i] - B[i]/(T_j[j] + C[i])) for j in range(len(T_j))] for i in range(3)])


In [10]:
A= [7.89750, 7.28610, 7.01050]
B= [1474.08, 1251.33, 1152.21]
C= [229.664, 229.664, 229.664]
T_j= [65, 98, 97]

print(calculate_Pij(A,B,C,T_j))



[[ 785.09243236 2504.63922632 2426.46128911]
 [1095.1321941  2931.95583891 2854.08359123]
 [1259.64939087 3119.33795651 3042.97046498]]


In [12]:
def calculatate_Kij(Pij_sat: np.array)-> np.array:
    """
    Calculate the Ki values for the components.

    Parameters:
    Pi_sat : list
        List of saturation pressures for the components.

    Returns:
    list
        List of Ki values for the components.
    """
    return [[Pij_sat[i][j] for i in range(len(Pij_sat))] for j in range(len(Pij_sat[0]))]

print(calculatate_Kij(calculate_Pij(A,B,C,T_j)))

[[785.0924323551616, 1095.132194103674, 1259.6493908746313], [2504.63922632249, 2931.9558389086556, 3119.3379565135674], [2426.4612891144425, 2854.0835912281036, 3042.97046498474]]


In [13]:
def thomas_algorithm( A: np.array, B: np.array, C: np.array, D: np.array)-> np.array:
    """
    Solve a tridiagonal system of equations using the Thomas algorithm.

    Parameters:
    A : list
        Lower diagonal elements.
    B : list
        Main diagonal elements.
    C : list
        Upper diagonal elements.
    D : list
        Right-hand side vector.

    Returns:
    list
        Solution vector.
    """
    n = len(B)
    c_prime = np.zeros(n)
    d_prime = np.zeros(n)
    x = np.zeros(n)
    
    c_prime[0] = C[0]/B[0]
    d_prime[0] = D[0]/B[0]
    
    for i in range(1, n):
        c_prime[i] = C[i]/(B[i] - A[i]*c_prime[i-1])
        d_prime[i] = (D[i] - A[i]*d_prime[i-1])/(B[i] - A[i]*c_prime[i-1])
    
    x[-1] = d_prime[-1]
    
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i]*x[i+1]
    
    return x

A =[0,100,100,200,200]
B =[-150,-344.5,-525.5,-605,-545]
C =[244.5,325.5,405,495,0]
D= [0,0,-30,0,0]

print(thomas_algorithm(A,B,C,D))

[0.56640823 0.34748971 0.19376154 0.09153758 0.03359177]


In [17]:
def check_summation( Xij: np.array) -> bool:
    """
    Check if the summation of Xij is equal to 1.

    Parameters:
    Xij : list
        List of component mole fractions.

    Returns:
    bool
        True if the summation is equal to 1, False otherwise.
    """
    return np.isclose(sum(Xij), 1, atol=1e-10)
print(check_summation([0.2, 0.3, 0.5]))

True


In [38]:
def calculate_Lj( feed_tray: int , Vj: np.array)-> np.array:
    """
    Calculate the liquid flow rates for each component.

    Parameters:
    Kij : list
        List of Ki values for the components.
    Vj : list
        List of vapor flow rates.

    Returns:
    list
        List of liquid flow rates for each component.
    """

    Lj = np.zeros_like(Vj)

    # Lj Before feed tray
    for i in range(feed_tray-1):
        Lj[i] = Vj[i+1] + 1
        
    # Lj After feed tray
    for i in range(feed_tray-1, 3-1):
        Lj[i] = Vj[i+1] + 2+ 1
        
    return Lj

In [39]:
calculate_Lj(2, [0, 0.3, 0.5])


array([1.3, 3.5, 0. ])

In [41]:
def calculate_Aj(Lj: np.array)-> np.array:
    """
    Calculate the Aj values for each component.

    Parameters:
    Lj : list
        List of liquid flow rates for each component.
    Vj : list
        List of vapor flow rates.

    Returns:
    list
        List of Aj values for each component.
    """
    # Aj = Lj-1
    Aj = np.zeros_like(Lj)
    Aj[0] = 0
    Aj[1:] = Lj[:-1]
    return Aj

In [42]:
calculate_Aj(calculate_Lj(2, [0, 0.3, 0.5]))

array([0. , 1.3, 3.5])

In [48]:
def calculate_Dj(feed_tray :int)-> np.array:
    """
    Calculate the Dj values for each component.

    Returns:
    list
        List of Dj values for each component.
    """
    # Dij = 0
    Dij = np.zeros((3, 3))
    Dij[feed_tray-1] = - 100 * np.array([0.6, 0.2, 0.2])
    return Dij

In [49]:
calculate_Dj(2)

array([[  0.,   0.,   0.],
       [-60., -20., -20.],
       [  0.,   0.,   0.]])

In [50]:
def thomas_algorithm(A: np.array, B: np.array, C: np.array, D: np.array)-> np.array:
    """
    Solve a tridiagonal system of equations using the Thomas algorithm.

    Parameters:
    A : list
        Lower diagonal elements.
    B : list
        Main diagonal elements.
    C : list
        Upper diagonal elements.
    D : list
        Right-hand side vector.

    Returns:
    list
        Solution vector.
    """
    n = len(B)
    c_prime = np.zeros(n)
    d_prime = np.zeros(n)
    x = np.zeros(n)
    
    c_prime[0] = C[0]/B[0]
    d_prime[0] = D[0]/B[0]
    
    for i in range(1, n):
        c_prime[i] = C[i]/(B[i] - A[i]*c_prime[i-1])
        d_prime[i] = (D[i] - A[i]*d_prime[i-1])/(B[i] - A[i]*c_prime[i-1])
    
    x[-1] = d_prime[-1]
    
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i]*x[i+1]
    
    return x

In [51]:
A = [0, 100, 100, 200, 200]
B = [-150, -344.5, -525.5, -605, -545]
C = [244.5, 325.5, 405, 495, 0]
D = [0, 0, -30, 0, 0]

thomas_algorithm(A, B, C, D)

array([0.56640823, 0.34748971, 0.19376154, 0.09153758, 0.03359177])

In [260]:
A = [0, 2000, 3000]
B = [-4088.4, -11024.2, -10751.2]
C = [8324.2,8351.2,0]
D = [0,-600, 0]

thomas_algorithm(A, B, C, D)

array([0.2643203 , 0.12981994, 0.03622478])

In [285]:
A = [0, 2000, 3000]
B = [-2921, -6847, -6662.4]
C = [5447,5262.4,0]
D = [0,-200, 0]

thomas_algorithm(A, B, C, D)

array([0.49869638, 0.26743017, 0.12042065])

In [286]:
A = [0, 2000, 3000]
B = [-2198.2, -5587.6, -5470.6]
C = [3187.6, 3070.6 ,0]
D = [0,-200, 0]

thomas_algorithm(A, B, C, D)

array([0.28899746, 0.19929546, 0.10929083])

In [582]:
from typing import Tuple

In [586]:
class Distillation:
    def __init__(self, feed_rate: int = 1000, distillate_rate: int = 600, reflux_rate: int = 2000, stages: int = 3, pressure: int = 1, components: list = ['methanol', 'ethanol', 'n-propanol'], feed_composition: list = [0.6, 0.2, 0.2], boiling_points: list = [65, 98, 97], convergence_tolerance: float = 1e-6, max_iterations: int = 1000):
        """ 
        Initialize the distillation column parameters.
        
        Parameters:
        feed_rate : int
            Feed rate of the mixture in kmol/h.
        distillate_rate : int
            Distillate rate in kmol/h.
        reflux_rate : int
            Reflux rate in kmol/h.
        stages : int
            Number of equilibrium stages.
        pressure : int
            Operating pressure in atm.
        components : list
            List of component names.
        feed_composition : list 
            List of feed composition in mole fractions.
        boiling_points : list
            List of boiling points for each component.
        temperature_profile : np.array
            Temperature profile for the distillation column.
        convergence_tolerance : float
            Convergence tolerance for the solution.
        max_iterations : int
            Maximum number of iterations for the solution.

        Returns:
        None
        """

        self.feed_rate = feed_rate
        self.distillate_rate = distillate_rate
        self.bottom_pdt_rate = feed_rate - distillate_rate
        self.reflux_rate = reflux_rate
        self.stages = stages
        self.pressure = pressure
        self.components = components
        self.num_components = len(components)
        self.feed_composition = feed_composition
        self.boiling_points = boiling_points
        self.convergence_tolerance = convergence_tolerance
        self.max_iterations = max_iterations
        self.temperature_profile = np.zeros((self.stages, len(self.components)))
        
    def calculate_Pij(self, C1: np.array, C2: np.array, C3: np.array, C4: np.array, C5: np.array, T_j: np.array) -> np.array:
        """
        Calculate the saturation pressure of a component at a given temperature.

        Parameters:
        C1, C2, C3, C4, C5 : np.array
            Coefficients for the equation.
        T_j : np.array
            Temperature in Celsius.

        Returns:
        np.array
            Saturation pressure in atm.
        """
       # Convert temperature from Celsius to Kelvin
        T_j_K = T_j + 273.15

        # 𝑃𝑠 = exp[𝐶1 + 𝐶2𝑇 + 𝐶3ln𝑇 + 𝐶4𝑇**𝐶5]

        P = np.array([
            [
                np.exp(C1[i] + C2[i] / T_j_K[j] + C3[i] * np.log(T_j_K[j]) + C4[i] * T_j_K[j] ** C5[i])
                for j in range(len(T_j))
            ] 
            for i in range(len(self.components))
        ])
        P = P*1e-5
        return P
    
    def calculatate_Kij(self, Pij_sat: np.array)-> np.array:
        """
        Calculate the Ki values for the components.

        Parameters:
        Pi_sat : list
            List of saturation pressures for the components.

        Returns:
        list
            List of Ki values for the components.
        """
        return np.array([[Pij_sat[i][j]/self.pressure for i in range(len(Pij_sat))] for j in range(len(Pij_sat[0]))])
    
    def calculate_Lj(self, feed_tray: int , Vj: np.array)-> np.array:
        """
        Calculate the liquid flow rates for each component.

        Parameters:
        feed_tray : int
            NUmber of tray at which feed is given in the distillation column.
        Vj : list
            List of vapor flow rates.

        Returns:
        list
            List of liquid flow rates for each component.
        """
        Lj = np.zeros_like(Vj)

        # Lj Before feed tray
        for i in range(feed_tray-1):
            Lj[i] = Vj[i+1] - self.distillate_rate
            
        # Lj After feed tray
        for i in range(feed_tray-1, self.stages-1):
            Lj[i] = Vj[i+1] + self.feed_rate - self.distillate_rate
        
        for i in range(self.stages-1,self.stages):
            Lj[i] = Lj[i-1]

        return Lj
    
    def calculate_Aj(self, Lj: np.array)-> np.array:
        """
        Calculate the Aj values for each component.

        Parameters:
        Lj : list
            List of liquid flow rates for each component.

        Returns:
        list
            List of Aj values for each component.
        """
        # Aj = Lj-1
        Aj = np.zeros_like(Lj)
        Aj[0] = 0
        Aj[1:] = Lj[:-1]
        return Aj
    
    def calculate_Bij(self,Vj: np.array, Lj: np.array, Kij: np.array)-> np.array:
        """
        Calculate the Bij values for each component.

        Parameters:
        Lj : list
            List of liquid flow rates for each component.
        Kij : list
            List of Ki values for the components.

        Returns:
        list
            List of Bij values for each component.
        """
        # Bij[i][j] = Lj[i] + (Vj+1[i] +self.reflux) * Kij[i][j]
        
        
        return np.array([[self.distillate_rate - Lj[j] - (Vj[j]) * Kij[j][i] for j in range(len(Kij[0]))] for i in range(len(Kij))])
    
    def calculate_Cij(self, Vj: np.array, Kij: np.array)-> np.array:
        """
        Parameters:
        Vj : list
            List of vapor flow rates.
        Kij : list
            List of Ki values for the components.

        Returns:
        list
            List of Cj values for each component.
        """
        # Cij = - Vj * Kij
        # Ensure Vj is reshaped for broadcasting
        Vj_reshaped = Vj.reshape(1, -1)  # Shape (1, m)

        # Compute Cij using broadcasting
        Cij = Vj_reshaped * Kij  # (n, m) * (1, m) results in (n, m)
        Cij = np.transpose(Cij)
        # Shift left and set the last column to 0
        Cij_shifted = np.hstack((Cij[:, 1:], np.zeros((Cij.shape[0], 1))))
        return Cij_shifted
    
    def calculate_Dij(self,  feed_tray :int)-> np.array:
        """
        Calculate the Dj values for each component.

        Returns:
        list
            List of Dj values for each component.
        """
        # Dj = 0.0
        # Dij = 0
        Dij = np.zeros((self.num_components, self.num_components))
        Dij[feed_tray-1] = [-self.feed_rate* X for X in self.feed_composition]
        return np.transpose(Dij)
    
    def calculate_Tj(self, A:np.array, B: np.array, C: np.array, Xij:np.array ) -> np.array:
        """
        Calculate the temperature profile for the distillation column.

        Parameters:
        A : list
            List of A coeff. of antoine equation.
        B : list
            List of B coeff. of antoine equation.
        C : list
            List of C coeff. of antoine equation.
        Xij : list
            List of Xij values for the components.

        Returns:
        list
            Temperature values for stages of the distillation column.
        """
        # returning that value of T_j for which summation of  

        # np.e(A[i] + (B[i]/T_j[j]+C[i])Xij[i][j])  

        # for all i and j values should be equal to self.pressure, 
        # for this we need to assume the values of T_j from the range of boiling points of the components for iteration

        T_j = np.zeros(self.stages)
        for stage in range(self.stages):
            T_guess = np.mean(self.boiling_points)
            for _ in range(self.max_iterations):
                P_total = sum([10**(A[i] - (B[i] / (T_guess + C[i]))) * Xij[stage][i] for i in range(len(self.components))])
                if np.isclose(P_total, self.pressure, atol=self.convergence_tolerance):
                    break
                T_guess += (self.pressure - P_total) * 0.1  # Adjust the temperature guess
            T_j[stage] = T_guess
        return T_j



    def check_summation(self, Xij: np.array) -> np.array:
        """
        Check if the summation of Xij is equal to 1.

        Parameters:
        Xij : list
            List of component mole fractions.

        Returns:
        bool
            True if the summation is equal to 1, False otherwise.
        """
        # NOrmalisation of Xij values to 1
        # Normalize each row to sum to 1
        Xij_nor = Xij / Xij.sum(axis=1, keepdims=True)  # Row-wise normalization

        # Check if each row sums to 1 within tolerance
        # is_normalized = np.all(np.isclose(Xij_nor.sum(axis=1), 1, atol=self.convergence_tolerance))

        return Xij_nor # Row-wise normalization
    
    # calculate Tj from summation of Xij and Kij values equals to 1
    def calculate_Tj(self, A:np.array, B: np.array, C: np.array, Xij:np.array ) -> np.array:
        """
        Calculate the temperature profile for the distillation column.

        Parameters:
        A : list
            List of A coeff. of antoine equation.
        B : list
            List of B coeff. of antoine equation.
        C : list
            List of C coeff. of antoine equation.
        Xij : list
            List of Xij values for the components.

        Returns:
        list
            Temperature values for stages of the distillation column.
        """
        # returning that value of T_j for which summation of  

        # np.e(A[i] + (B[i]/T_j[j]+C[i])Xij[i][j])  

        # for all i and j values should be equal to self.pressure, 
        # for this we need to assume the values of T_j from the range of boiling points of the components for iteration

        T_j = np.zeros(self.stages)
        for stage in range(self.stages):
            T_guess = np.mean(self.boiling_points)
            for _ in range(self.max_iterations):
                P_total = sum([10**(A[i] - (B[i] / (T_guess + C[i]))) * Xij[stage][i] for i in range(len(self.components))])
                if np.isclose(P_total, self.pressure, atol=self.convergence_tolerance):
                    break
                T_guess += (self.pressure - P_total) * 0.1  # Adjust the temperature guess
            T_j[stage] = T_guess
        return T_j
    
    def calculate_Vj_heat_balance(self, Lj: np.array, hL: np.array, hV: np.array, hF: float, Q: float, feed_tray: int) -> np.array:
        """
        Calculate the vapor flow rates for each component using the heat balance equation.

        Parameters:
        Lj : np.array
            List of liquid flow rates for each component.
        hL : np.array
            List of enthalpies of liquid for each stage.
        hV : np.array
            List of enthalpies of vapor for each stage.
        hF : float
            Enthalpy of the feed.
        Q : float
            Heat added or removed from the stage.
        feed_tray : int
            The feed tray number.

        Returns:
        np.array
            Vapor flow rates for each component.
        """
        Vj = np.zeros_like(Lj)

        # Heat balance equation for each stage
        for j in range(self.stages):
            if j == 0:
                Vj[j] = (hL[j] * Lj[j] + Q) / hV[j]
            elif j == self.stages - 1:
                Vj[j] = (hL[j-1] * Lj[j-1] + hF * self.feed_rate - hL[j] * Lj[j] + Q) / hV[j]
            else:
                Vj[j] = (hL[j-1] * Lj[j-1] + hV[j+1] * Vj[j+1] + (hF * self.feed_rate if j == feed_tray else 0) - hL[j] * Lj[j] + Q) / hV[j]

        return Vj
    
    # Calcualtion of enthalpy of liquid and vapor and feed for each stage
    def calculate_enthalpies(self, Xij: np.array, Kij: np.array, hij: np.array) -> Tuple[np.array, np.array, float]:
        """
        Calculate the enthalpies of liquid, vapor, and feed.

        Parameters:
        Xij : np.array
            Mole fractions of components in the liquid phase for each stage.
        Kij : np.array
            Equilibrium constants for the components.
        hij : np.array
            Enthalpies of components.

        Returns:
        hL : np.array
            Enthalpies of liquid for each stage.
        hV : np.array
            Enthalpies of vapor for each stage.
        hF : float
            Enthalpy of the feed.
        """
        hL = np.zeros(self.stages)
        hV = np.zeros(self.stages)

        for j in range(self.stages):
            hL[j] = sum([Xij[j][i] * hij[i] for i in range(len(self.components))])
            hV[j] = sum([Kij[j][i] * Xij[j][i] * hij[i] for i in range(len(self.components))]) / sum([Kij[j][i] * Xij[j][i] for i in range(len(self.components))])
        
        hF = sum([self.feed_composition[i] * hij[i] for i in range(len(self.components))])
        
        return hL, hV, hF

In [587]:
C1 = np.array([81.768, 74.475, 88.134])
C2 = np.array([-6876, -7164.3, -8438.6])
C3 = np.array([-8.7078, -7.327, -9.0766])
C4 = np.array([7.1926e-6, 3.1340e-6, 8.3303e-18])
C5 = np.array([2, 2, 6])
T_j = np.array([65, 98, 97])

In [588]:
distillation = Distillation()
print(distillation.calculate_Pij(C1, C2, C3, C4, C5, T_j))
P_ij = distillation.calculate_Pij(C1, C2, C3, C4, C5, T_j)

[[1.03413114 3.3169486  3.21284295]
 [0.58565375 2.09559761 2.02411882]
 [0.30753875 1.22607848 1.18120467]]


In [589]:
print(distillation.calculatate_Kij(Pij_sat=P_ij))
K_ij = distillation.calculatate_Kij(Pij_sat=P_ij)

[[1.03413114 0.58565375 0.30753875]
 [3.3169486  2.09559761 1.22607848]
 [3.21284295 2.02411882 1.18120467]]


In [590]:
print(distillation.calculate_Lj(feed_tray=2, Vj=[2600, 2600,2600]))

L_j = distillation.calculate_Lj(feed_tray=2, Vj=[2600, 2600,2600])

[2000 3000 3000]


In [591]:
A_j = distillation.calculate_Aj(Lj=L_j)
print(A_j)

[   0 2000 3000]


In [592]:
B_ij = distillation.calculate_Bij(Lj=L_j, Kij=K_ij,Vj=[2600, 2600,2600])
print(B_ij)

[[ -4088.7409626  -11024.06634939 -10753.3916643 ]
 [ -2922.69974954  -7848.55378498  -7662.70892353]
 [ -2199.60074655  -5587.8040587   -5471.1321516 ]]


In [593]:
C_ij = distillation.calculate_Cij(Vj=np.array([2600, 2600, 2600]), Kij=K_ij)
print(C_ij)

[[8624.06634939 8353.3916643     0.        ]
 [5448.55378498 5262.70892353    0.        ]
 [3187.8040587  3071.1321516     0.        ]]


In [594]:
D_ij = distillation.calculate_Dij(feed_tray=2)
print(D_ij)

[[   0. -600.    0.]
 [   0. -200.    0.]
 [   0. -200.    0.]]


In [595]:
print(A_j)
print(B_ij)
print(C_ij)
print(D_ij)

[   0 2000 3000]
[[ -4088.7409626  -11024.06634939 -10753.3916643 ]
 [ -2922.69974954  -7848.55378498  -7662.70892353]
 [ -2199.60074655  -5587.8040587   -5471.1321516 ]]
[[8624.06634939 8353.3916643     0.        ]
 [5448.55378498 5262.70892353    0.        ]
 [3187.8040587  3071.1321516     0.        ]]
[[   0. -600.    0.]
 [   0. -200.    0.]
 [   0. -200.    0.]]


In [596]:
temp_values = []

for i in range(3):
    temp = thomas_algorithm(A_j, B_ij[i], C_ij[i], D_ij[i])
    print(temp)
    temp_values.append(temp)


[0.28278968 0.13407292 0.0374039 ]
[0.18101588 0.09710009 0.03801531]
[0.28833273 0.19895102 0.10909133]


In [597]:
Xij =np.array(temp_values)
Xij

array([[0.28278968, 0.13407292, 0.0374039 ],
       [0.18101588, 0.09710009, 0.03801531],
       [0.28833273, 0.19895102, 0.10909133]])

In [598]:
Xij_= distillation.check_summation(Xij)

In [599]:
Xij_

array([[0.62251934, 0.29514155, 0.08233911],
       [0.57259718, 0.30715116, 0.12025166],
       [0.48347548, 0.3336005 , 0.18292402]])

In [600]:
A = [5.20409, 5.3229, 5.31384]
B = [1581.341, 1670.409, 1690.864]
C = [-33.5, -40.191, -51.804]

In [601]:
Tj_new = distillation.calculate_Tj(A,B,C,Xij_)

In [602]:
Tj_new

array([186.66663749, 186.66663967, 186.66664353])

In [613]:
hij_v = [35300, 38600, 47600]

In [614]:
hv,hl,hf = distillation.calculate_enthalpies(Xij_,K_ij,hij=hij_v)

In [615]:
Vj_new = distillation.calculate_Vj_heat_balance(L_j,hl,hv,hf,Q=0,feed_tray=2)

In [616]:
Vj_new

array([1949, -994,  952])

In [ ]:
hij_v = [-35.3, -38.6, -47.6]

In [ ]:
import numpy as np
from typing import Tuple

class MultistageDistillation:
    def __init__(self, components, boiling_points, pressure, stages, max_iterations, convergence_tolerance, feed_rate, feed_composition, reflux, distillate):
        self.components = components
        self.boiling_points = boiling_points
        self.pressure = pressure
        self.stages = stages
        self.max_iterations = max_iterations
        self.convergence_tolerance = convergence_tolerance
        self.feed_rate = feed_rate
        self.feed_composition = feed_composition
        self.reflux = reflux
        self.distillate = distillate

    def calculate_Tj(self, A: np.array, B: np.array, C: np.array, Xij: np.array) -> np.array:
        """
        Calculate the temperature profile (Tj) for each stage in the distillation column 
        using the Bubble Point method.

        Parameters:
        A, B, C : Antoine equation coefficients (arrays)
        Xij : Mole fractions of components in the liquid phase for each stage.

        Returns:
        T_j : np.array
            Temperature values for each stage.
        """
        T_j = np.zeros(self.stages)

        for stage in range(self.stages):
            T_guess = np.mean(self.boiling_points)  # Initial guess within boiling points range

            for _ in range(self.max_iterations):
                P_total = sum([10 ** (A[i] - (B[i] / (T_guess + C[i]))) * Xij[stage][i] 
                               for i in range(len(self.components))])

                if np.isclose(P_total, self.pressure, atol=self.convergence_tolerance):
                    break
                
                T_guess += (self.pressure - P_total) * 0.1  # Adjust the temperature guess

            T_j[stage] = T_guess

        return T_j

    def calculate_Vj_heat_balance(self, Lj: np.array, hL: np.array, hV: np.array, hF: float, Q: float, feed_tray: int) -> np.array:
        """
        Calculate vapor flow rates (Vj) for each stage using the Heat Balance Equation.

        Parameters:
        Lj : np.array
            Liquid flow rates per stage.
        hL : np.array
            Enthalpies of the liquid phase per stage.
        hV : np.array
            Enthalpies of the vapor phase per stage.
        hF : float
            Enthalpy of the feed.
        Q : float
            Heat added or removed from the system.
        feed_tray : int
            Index of the feed tray.

        Returns:
        Vj : np.array
            Vapor flow rates per stage.
        """
        Vj = np.zeros(self.stages)

        for j in range(self.stages):
            if j == 0:  # Condenser stage
                Vj[j] = (hL[j] * Lj[j] + Q) / hV[j]
            elif j == self.stages - 1:  # Reboiler stage
                Vj[j] = (hL[j-1] * Lj[j-1] + hF * self.feed_rate - hL[j] * Lj[j] + Q) / hV[j]
            else:  # Intermediate stages
                Vj[j] = (hL[j-1] * Lj[j-1] + hV[j+1] * Vj[j+1] + (hF * self.feed_rate if j == feed_tray else 0) - hL[j] * Lj[j] + Q) / hV[j]

        return Vj

    def calculate_enthalpies(self, Xij: np.array, Kij: np.array, hij: np.array) -> Tuple[np.array, np.array, float]:
        """
        Calculate the enthalpies of liquid (hL), vapor (hV), and feed (hF).

        Parameters:
        Xij : np.array
            Mole fractions of components in the liquid phase per stage.
        Kij : np.array
            Equilibrium constants per component per stage.
        hij : np.array
            Pure component enthalpies.

        Returns:
        Tuple containing:
        - hL : np.array (Liquid enthalpies per stage)
        - hV : np.array (Vapor enthalpies per stage)
        - hF : float (Feed enthalpy)
        """
        hL = np.zeros(self.stages)
        hV = np.zeros(self.stages)

        for j in range(self.stages):
            hL[j] = sum([Xij[j][i] * hij[i] for i in range(len(self.components))])
            hV[j] = sum([Kij[j][i] * Xij[j][i] * hij[i] for i in range(len(self.components))]) / \
                    sum([Kij[j][i] * Xij[j][i] for i in range(len(self.components))])

        hF = sum([self.feed_composition[i] * hij[i] for i in range(len(self.components))])

        return hL, hV, hF
